# File: exercise_0.ipynb
 - Created: 4 June 2013 Harrison B. Prosper, INFN SOS 2013, Vietri sul Mare, Italy
 - Adapted for CMSDAS 2016, LPC Fermilab, Harrison B. Prosper
 - Adapted for CMSDAS 2019, LPC Fermilab, Javier Duarte
 - Adapted for CMSDAS 2024, LPC Fermilab, Harrison B. Prosper
 - Adapted for CMSDAS 2026, LPC Fermilab, Mohammad W., Vasilije P.
 
## Description

This notebook is a compact, end-to-end tutorial on using **PyROOT** and **RooFit/RooStats** to build a simple model, generate toy data, fit it, and evaluate fit quality. The emphasis is on learning the *RooFit object model* (variables, PDFs, datasets, fit results) and the *workflow patterns* that show up repeatedly in CMS statistics (workspace-driven definitions, likelihood fits, binned/unbinned views of the same data).

We use a one-dimensional observable $x$ and a double-exponential PDF with parameters $\theta \equiv (a,b,c)$. The notebook then follows the standard pipeline:

- **Workspace & factory syntax:** create and store the observable, parameters, PDFs, and datasets in a `RooWorkspace` so the full model lives in one consistent container.
- **Toy generation:** generate an unbinned `RooDataSet` of size $T$ directly from the PDF. This provides a controlled environment to validate the fitting and GOF machinery.
- **Unbinned fit (Part 1):** fit the PDF to the unbinned dataset with maximum likelihood, save the `RooFitResult`, and inspect parameter values, uncertainties, and correlations.
- **Binned fit (Part 2):** bin the same events into a `RooDataHist` with $M$ bins and repeat the fit using a multinomial likelihood (fixed total count). This mirrors the logic of histogram-based fits and templates.
- **Goodness-of-fit (Part 3):** compute two binned GOF measures and convert them to **p-values** using asymptotic $\chi^2$ tail probabilities, giving a quantitative answer to: “are the bin-to-bin deviations typical if the fitted model is true?”


ROOT (RooFit/RooStats) documentation: https://root.cern.ch/


### Some useful shortcuts

 - Use **esc r** to disable a cell
 - Use **esc y** to reactivate it
 - Use **esc m** to go to markdown mode. Markdown is the typesetting language used in jupyter notebooks. In a markdown cell, double tap the mouse or glide pad (on your laptop) to go to edit mode.
 - Shift + return to execute a cell (including markdown cells).
 - If equations don't typeset, try double tapping the cell again, and re-execute the cell.

In [ ]:
# --- Standard Python utilities ---
import sys                 # Python runtime utilities (paths, env, argv, etc.)
from time import sleep     # simple pauses for pacing/demo (optional)
import math                # basic math functions (log, sqrt, etc.) used later

# --- ROOT / RooFit via PyROOT ---
import ROOT                # gives access to ROOT + RooFit/RooStats classes from Python

# --- Local helper utilities for this tutorial ---
# Add the CMSDAS helper module directory so we can import plotting style helpers.
sys.path.append('../python')
from histutil import setStyle, mkhist1   # consistent ROOT plotting style + quick 1D histogram helper

# RooFit can be very chatty (INFO/WARNING messages during fitting).
# For this short exercise we suppress everything below FATAL to keep output readable.
# If anything looks wrong (unexpected fit behavior, NaNs, crashes), comment out the kill line
# to re-enable RooFit diagnostics and see what it’s complaining about.
msgservice = ROOT.RooMsgService.instance()
msgservice.setGlobalKillBelow(ROOT.RooFit.FATAL)

## Create workspace

In [ ]:
# RooWorkspace = central container for RooFit/RooStats objects (x, parameters, PDFs, data).
# Using its factory() lets us define the model with concise strings and reuse/save everything consistently.
wspace = ROOT.RooWorkspace("CMSDAS")

## Define the model ingredients: **observable** and **parameters**

In this exercise we fit a *double-exponential* PDF to data.

- **Observable**: $x$ (the quantity we measure and histogram/fit)
- **Parameters**: $a, b, c$ (numbers the fit will determine)

In RooFit, *everything is an object* (variables, PDFs, datasets, fit results).  

A convenient way to create variables is the workspace **factory()**, which parses a short string:

```python
    wspace.factory(f"{name}[{initial_value}, {xmin}, {xmax}]")
```

In [ ]:
# ------------------------------------------------------------
# Observable definition (given)
# ------------------------------------------------------------
# We'll fit the PDF over x ∈ [0, 20]. Keeping the range explicit
# helps later when we integrate the PDF over bins (GOF section).
xmin, xmax = 0.0, 20.0
wspace.factory(f"x[0, {xmin}, {xmax}]")

# Choose the number of bins used for (i) display and (ii) the binned fit.
# (This does not affect the unbinned likelihood fit we'll do later.)
M = 15
wspace.var("x").setBins(M)

# ------------------------------------------------------------
# EXERCISE: define the model parameters a, b, c
# ------------------------------------------------------------
# Create THREE RooRealVar parameters in the workspace using factory():
#
#   a : initial value = 0.4, range = [0.0, 1.0]
#   b : initial value = 3.0, range = [0.01, 20.0]
#   c : initial value = 9.0, range = [0.01, 20.0]
#
# Notes:
# - a is a mixture fraction, so it must stay between 0 and 1.
# - b and c are exponential scale parameters, so they must be > 0.
#
# TODO: write three lines like:
#   wspace.factory("a[ ... ]")
#   wspace.factory("b[ ... ]")
#   wspace.factory("c[ ... ]")
# ------------------------------------------------------------

parameters = ["a", "b", "c"]
P = len(parameters)   # number of floating parameters in the fit

## Define double-exponential PDF model

The model to be fitted, called `model`, is defined by the probability density function (pdf)
$$p(x|a, b, c) = a \exp(-x/b)/b + (1-a) \exp(-x/c)/c$$

In RooFit we can build this pdf in a few equivalent ways. The goal is the same in each case:
create a pdf named `model` that depends on the observable `x` and parameters `a, b, c`.


### A) Directly with the workspace factory (string expression)

RooFit provides `RooGenericPdf` for simple analytic forms. The factory syntax is:

```python
  GenericPdf::<user-defined-name>("<function>", {...})
```

* We drop the `"Roo"` prefix (so `RooGenericPdf` → `GenericPdf`).
* The braces `{...}` specify the variable list (internally a `RooArgList`) that appear in the expression.

Example (create a pdf named `model`):

```python
  wspace.factory('GenericPdf::model("a*exp(-x/b)/b + (1-a)*exp(-x/c)/c", {x,a,b,c})')
```


### B) Define the function in C++ (inline), then call it from `GenericPdf`

This is useful when the function gets longer, you want to reuse it, or you want C++ compilation/typing.

```python
  ROOT.gInterpreter.Declare(
      """
  #include <cmath>
  double dbexp(double x, double a, double b, double c)
  {
    return a*exp(-x/b)/b + (1-a)*exp(-x/c)/c;
  }
  """
  )
  from ROOT import dbexp

  wspace.factory('GenericPdf::model("dbexp(x,a,b,c)", {x,a,b,c})')
```


### C) Load the C++ function from a `.cc` file (same idea as B, but external file)

If you already have a function in a standalone file:

```python
  gROOT.ProcessLine(open("<function>.cc").read())
  from ROOT import <function>
```

Notes:

* `open(...).read()` passes the whole file as a string to ROOT’s JIT compiler.
* If additional headers/libs are needed, set include/lib paths *before* compiling:

```python
  gSystem.AddIncludePath("-I<path1> ...")
  gSystem.AddLinkedLibs("-L<libdir> -l<library> ...")
```

In [ ]:
# ------------------------------------------------------------
# EXERCISE: build the PDF named "model" in the workspace
# ------------------------------------------------------------
# Create the double-exponential pdf using ONE of the methods described above:
#
#   (A) Direct RooGenericPdf expression via wspace.factory(...)
#   (B/C) Use a C++ helper function and call it inside GenericPdf
#         (a file "models.cc" is provided; compile/load it, then call dbexp(x,a,b,c))
#
# Requirement: after this block, the workspace must contain a pdf named "model".
#
# TODO: implement your chosen method here.
# ------------------------------------------------------------

# After you have created it in the workspace, retrieve it as a Python handle:
model = wspace.pdf("model")

## Generate a toy dataset from the model

Before we can fit anything, we need *data*. In this exercise we create **toy (pseudo-)data** by *sampling* events directly from our PDF `model`. Concretely, `generate()` uses the PDF $p(x\mid a,b,c)$ as a probability law and returns a list of random draws $x_1,\dots,x_T$ distributed according to that law.

Why do this?

- It gives a controlled, end-to-end test of the full workflow:  
  **define model → generate toy data → fit (unbinned/binned) → assess goodness-of-fit**.
- Because we know the *truth model* used to generate the sample, we can sanity-check that the fit recovers sensible parameter values (up to statistical fluctuations).
- It mirrors how we validate statistical procedures in CMS: toys are a lightweight way to stress-test fits, uncertainties, and GOF behavior before moving on to real data and more realistic models.

RooFit objects and commands used here

- **`defineSet("obs", "x")`**: define a *named set* of observables in the workspace (here $x$).  
  Many RooFit methods (including `generate`) take a **set** of observables (a `RooArgSet`), even if there is only one variable.
- **`set("obs")`**: retrieve that named set from the workspace as a Python handle.
- **`model.generate(obs, T)`**: generate an **unbinned** `RooDataSet` with `T` events, i.e. a collection of $T$ sampled $x$-values distributed as $p(x\mid a,b,c)$.

In [ ]:
# ------------------------------------------------------------
# EXERCISE: define observables and generate toy data
# ------------------------------------------------------------

# 1) Define the set obs = (x) inside the workspace
#    Syntax: wspace.defineSet("<setName>", "<comma-separated variable names>")
# TODO: wspace.defineSet("obs", "x")

# 2) Retrieve the set as a Python handle
#    Syntax: obs = wspace.set("<setName>")
# TODO: obs = wspace.set("obs")

# 3) Generate an unbinned toy dataset from the PDF
T = 400  # number of events to generate
# TODO: data = model.generate(obs, T)

## Part 1: Unbinned maximum-likelihood fit

You now have an **unbinned** dataset `data` generated from your PDF `model`. In this part you will fit the parameters $\theta=(a,b,c)$ by **maximum likelihood** using the *unbinned* likelihood,
$$
\mathcal{L}(\theta) = \prod_{i=1}^{T} p(x_i\mid \theta),
\qquad
\text{equivalently minimize } \; -\ln\mathcal{L}(\theta).
$$

“Unbinned” means we use the individual event values $x_i$ directly (no histogramming). This is typically the most statistically efficient use of the information in the sample.

What RooFit does for you
When you call `fitTo`, RooFit:
- builds the negative log-likelihood from your PDF and dataset,
- runs the minimizer (typically Minuit/Minuit2) to find best-fit values $\hat{\theta}$,
- estimates parameter uncertainties and correlations from the curvature near the minimum,
- and returns a `RooFitResult` so you can inspect or reuse the fit outcome.

Key commands / objects

- **`ROOT.TStopwatch()`**: simple timing utility (`Start()`, `RealTime()`) so you can compare runtimes (useful later when you contrast unbinned vs binned fits or change $T$).
- **`model.fitTo(data, ROOT.RooFit.Save())`**: perform the unbinned ML fit and **return** a `RooFitResult` (using `Save()` so the result is not lost).
- **`results.Print()`**: print best-fit parameter values, uncertainties, and fit-status information.
- **`results.correlation(p1, p2)`**: correlation coefficient between two fitted parameters (helps diagnose degeneracies, e.g. between the two exponential scales).

In [ ]:
# ------------------------------------------------------------
# PART 1: unbinned fit of model to generated data
# ------------------------------------------------------------
print("=" * 80)
print("\t\tunbinned fit to toy data")
print("=" * 80)

# 1) Start a stopwatch to time the fit
# TODO:
# swatch = ROOT.TStopwatch()
# swatch.Start()

# 2) Perform the unbinned fit and SAVE the fit result
#    Syntax: results = model.fitTo(data, ROOT.RooFit.Save())
# TODO: results = model.fitTo(data, ROOT.RooFit.Save())

# 3) Print timing information
# TODO: print("real time: {:.3f} s".format(swatch.RealTime()))

# 4) Print the fit result summary (parameter values, errors, etc.)
# TODO:
# print("=" * 80)
# results.Print()

# 5) Print the correlation matrix for the fitted parameters
#    Hint: results.correlation("a","b") returns the correlation coefficient.
# TODO:
# print("\tcorrelation matrix")
# header = "{:>10s}".format("")
# for p in parameters:
#     header += "{:>10s}".format(p)
# print(header)
#
# for p1 in parameters:
#     row = "{:>10s}".format(p1)
#     for p2 in parameters:
#         row += "{:10.3f}".format(results.correlation(p1, p2))
#     print(row)

## Plot the fitted model over the data

After the unbinned fit, we want a quick visual check: plot the **dataset** and overlay the **fitted PDF** as a function of the observable $x$.

RooFit plotting is done through a **frame** object:
- **`xframe = x.frame()`**: create a `RooPlot` frame for the variable $x$ (this is the canvas “staging area” where curves/points get added).
- **`data.plotOn(xframe)`**: add the dataset points (with Poisson error bars) to the frame.
- **`model.plotOn(xframe)`**: add the PDF curve (using the *current* parameter values, i.e. the post-fit ones).
- **`model.paramOn(xframe)`**: add a small parameter box to the frame (best-fit values and errors).
- **`xframe.Draw()`**: draw everything currently stored on the frame.
- **`TCanvas(...); SaveAs(...)`**: standard ROOT canvas creation and file output.

In [ ]:
# ------------------------------------------------------------
# Plot results of the unbinned fit
# ------------------------------------------------------------

# 1) Create a RooPlot frame for the observable x
#    Syntax: xframe = wspace.var("x").frame()
# TODO:
# xframe = wspace.var("x").frame()

# 2) Optional: adjust the y-axis range for readability
#    (These numbers depend on T and your binning/plot settings.)
# TODO:
# xframe.SetMinimum(0.0)
# xframe.SetMaximum(100.0)

# 3) Add the dataset points to the frame
#    Syntax: data.plotOn(xframe)
# TODO:
# data.plotOn(xframe)

# 4) Overlay the fitted model curve on the same frame
#    Syntax: model.plotOn(xframe)
# TODO: model.plotOn(xframe)

# 5) Add a parameter box to the plot
#    Syntax: model.paramOn(xframe)
# TODO: model.paramOn(xframe)

# 6) Draw on a canvas and save
setStyle()
c1 = ROOT.TCanvas("fig_unbinnedFit", "fit", 600, 500)

# TODO:
# xframe.Draw()
# c1.Draw()
# c1.SaveAs("fig_unbinnedFit.png")

## PART 2: Do a binned fit

So far you fit `model` to the **unbinned** dataset `data` (a list of event values $x_i$). Now you will bin the *same* events into a `RooDataHist` and fit using **binned** likelihoods.

`RooDataHist` is RooFit’s histogram-like dataset. It is defined with respect to a set of observables (here `obs = {x}`), and it uses the binning attached to `x` (set earlier via `x.setBins(M)`). After binning, the data are summarized by counts $N_i$ in each bin $i=1,\dots,M$, with total
$$
T \equiv \sum_{i=1}^{M} N_i.
$$

Your PDF $p(x\mid a,b,c)$ is a *density*. To compare to binned counts, convert it into bin probabilities
$$
p_i(a,b,c) \equiv \int_{\text{bin }i} p(x\mid a,b,c)\,dx,
\qquad \sum_{i=1}^{M} p_i = 1,
$$
and (given the observed total $T$) the corresponding expected bin counts
$$
n_i(a,b,c) \equiv T\,p_i(a,b,c).
$$

There are two standard likelihood models for binned counts. The difference is whether you **model the total rate** (overall normalization) in addition to the **shape** across bins.

**A) Multinomial (non-extended; condition on the observed total $T$)**  
Here you treat $T$ as fixed *by construction* and fit only the shape parameters $(a,b,c)$ through the bin fractions $p_i$:
$$
\mathcal{L}_{\text{multi}}(a,b,c)
=
\frac{T!}{N_1!\cdots N_M!}\prod_{i=1}^{M} p_i(a,b,c)^{N_i}.
$$
Use this when you explicitly want to remove any information from the overall event count and constrain the fit using only how events are distributed in $x$ (i.e. a **shape-only** fit). In this exercise, because the dataset was generated with a fixed size $T$, the multinomial likelihood is the most direct binned analogue of the unbinned shape fit.

**B) Extended likelihood (binned product of Poissons; fit rate + shape)**  
Here you introduce a yield parameter $\nu$ (the expected total number of events in the fitted range) and model each bin count as Poisson with mean
$$
\mu_i(\nu,a,b,c) \equiv \nu\,p_i(a,b,c),
$$
so that
$$
\mathcal{L}_{\text{ext}}(\nu,a,b,c)
=
\prod_{i=1}^{M}\text{Pois}\!\left(N_i \mid \mu_i(\nu,a,b,c)\right)
=
\prod_{i=1}^{M}\frac{\mu_i^{N_i}e^{-\mu_i}}{N_i!}.
$$
This likelihood uses **both**:  
- the *shape* information through the $p_i(a,b,c)$, and  
- the *rate* information through how well $\nu$ predicts the observed total $T$ (since $\sum_i \mu_i = \nu$).

Use this when the absolute normalization carries physics information or constraints (e.g. signal strength, expected yields from luminosity$\times$cross section$\times$acceptance, or when comparing components whose relative normalizations are parameters). In real analyses, this is the default when you want the fit to learn from *both* the histogram shape and the total event count.

### RooFit switch
RooFit selects between these with **extended mode**:
- `Extended(False)` → multinomial (condition on $T$; shape-only)
- `Extended(True)`  → extended likelihood (introduce/floating yield $\nu$; rate + shape)

In [ ]:
# ------------------------------------------------------------
# PART 2: Bin the data (RooDataHist) and do a binned fit
# ------------------------------------------------------------

# EXERCISE (uncomment each TODO line when you're ready):
#
# 1) Build a binned dataset (RooDataHist) from the observable set "obs".
#    Syntax:
#      ROOT.RooDataHist(<name>, <title>, <RooArgSet_of_observables>)
#
# 2) Fill it by binning the unbinned dataset "data".
#    Hint: RooDataHist.add(...) will read events from a RooDataSet and increment bins.
#
# 3) Print a verbose summary so you can verify:
#    - which observable(s) are binned (here: x),
#    - the binning (taken from x.setBins(M)),
#    - the resulting bin contents.
#
# 4) Fit the existing PDF "model" to the binned data and store the RooFitResult.
#    Save() returns a RooFitResult object you can print/inspect.
#    Extended(False) conditions on the observed total count (multinomial).
#    (Optional later: switch to Extended(True) for an extended / multi-Poisson-style fit.)

# 1) Create the binned dataset container
# TODO: hdata = ROOT.RooDataHist("hdata", "binned data", obs)

# 2) Bin the unbinned events into hdata
# TODO: hdata.add(data)

# 3) Inspect the binned dataset
# TODO: print("=" * 40)
# TODO: hdata.Print("verbose")
# TODO: print("=" * 40)

# 4) Perform the binned fit
# TODO: results2 = model.fitTo(hdata, ROOT.RooFit.Save(), ROOT.RooFit.Extended(False))

# Inspect the fit result
# TODO: results2.Print()

In [ ]:
# ------------------------------------------------------------
# Plot the binned-fit result
# ------------------------------------------------------------
# Visualize the binned dataset (hdata) and overlay the fitted PDF (model)
# on a new RooPlot frame, so it doesn't overwrite the unbinned plot.

# Create a fresh canvas for the binned-fit plot
# TODO: c2 = ROOT.TCanvas("fig_binnedFit", "binned fit", 500, 500)

# Create a fresh RooPlot frame for x
# TODO: xframe2 = wspace.var("x").frame()

# Optional: set a visible y-axis range for this plot (adjust as needed)
# TODO: xframe2.SetMinimum(0)
# TODO: xframe2.SetMaximum(100)

# Plot the binned data and overlay the fitted model
# TODO: hdata.plotOn(xframe2)
# TODO: model.plotOn(xframe2)

# Optional: print parameter values/uncertainties on the plot
# TODO: model.paramOn(xframe2)

# Render + save
# TODO: xframe2.Draw()
# TODO: c2.Draw()
# TODO: c2.SaveAs(".png")

# Separator for notebook output
# TODO: print("=" * 80)

## PART 3: Do a bit of statistics (goodness-of-fit and p-values)

So far we have generated $T$ events, then fit the double-exponential model both unbinned and binned (with $M$ bins). The fit returns best-fit parameters $\hat{\theta}$ (here $\theta=(a,b,c)$), but we still need to check whether the fitted model provides an acceptable description of the *binned* data.

**Goodness-of-fit (gof) question:**  
        ```If the fitted model were true, how surprising are the observed bin-to-bin deviations?```

### Notation
- $M$: number of $x$ bins  
- $T$: total event count  
- $N_i$: observed count in bin $i$  
- $n_i$: expected (mean) count in bin $i$ under the fitted model,
  $$
  n_i \;=\; T \int_{\text{bin }i} p(x \mid \hat{\theta})\,dx .
  $$

Because $p(x\mid\hat{\theta})$ is a density, we integrate it over each bin to predict counts. That is why we build a RooFit integral over the current bin range.


### Two gof statistics
We compute two standard measures, both evaluated at $\hat{\theta}$:

**1) Pearson chi-square**
$$
X \;=\; \sum_{i=1}^{M} \frac{(N_i - n_i)^2}{n_i}.
$$

**2) Likelihood-ratio (deviance)**
$$
Y \;=\; -2 \log \left[\frac{p(\boldsymbol{N}\mid \boldsymbol{n})}{p(\boldsymbol{N}\mid \boldsymbol{N})}\right].
$$
Here $\boldsymbol{N}=(N_1,\dots,N_M)$ and $\boldsymbol{n}=(n_1,\dots,n_M)$. The denominator is the “saturated” model (perfect bin-by-bin fit), so $Y$ measures how much worse our fitted model is than a perfect description of the histogram.

**Practical note:** we restrict to bins with $N_i>5$ to stay in the regime where asymptotic approximations are more reliable.


### Degrees of freedom and p-values
Let $P$ be the number of fitted parameters (here $P=3$), and let $K$ be the number of retained bins (after applying $N_i>5$).

Assuming asymptotia and $H_0$ (“the model is correct”), both $X$ and $Y$ are approximately $\chi^2$-distributed with
$$
\text{ndf} \;=\; K - P - 1,
$$
where the “$-1$” reflects the fixed-total constraint $\sum_i N_i = T$ (multinomial conditioning).

For either statistic $Z\in\{X,Y\}$, the p-value is the upper-tail probability
$$
p\text{-value} \;=\; \Pr\!\left(Z \ge Z_{\text{obs}} \mid H_0\right).
$$
Small p-values suggest the observed deviations are unlikely under the model; large p-values suggest they are consistent with statistical fluctuations.

In the code we compute these tails with `ROOT.TMath.Prob(Z, ndf)`.

In [ ]:
# ------------------------------------------------------------
# PART 3: Goodness-of-fit on binned data (X, Y, and p-values)
# ------------------------------------------------------------
# Goal: compare observed bin counts N_i to expected counts n_i
# from the fitted PDF. Since a PDF is a *density*, we must
# integrate it over each x-bin to predict n_i.
#
# EXERCISE:
# 1) Build a normalized integral of the PDF over a named range "x-bin".
#    - Create a normalization set: ROOT.RooFit.NormSet(obs)
#    - Define a named x-range: wspace.var("x").setRange("x-bin", lo, hi)
#    - Create the integral: model.createIntegral(obs, normSet, ROOT.RooFit.Range("x-bin"))
#
# 2) Loop over bins to compute n_i = T * (integral over bin i), then accumulate:
#      X = Σ (N_i - n_i)^2 / n_i
#      Y = -2 Σ N_i ln(n_i / N_i)
#    Use only bins with N_i > 5 (asymptotic regime).
#
# 3) Convert X and Y to p-values using χ^2 tail probabilities:
#      ndf = K - P - 1   (K = number of bins used, P = #fit parameters)
#      p = ROOT.TMath.Prob(Z, ndf)

# 1) Normalized integral over a named bin-range
# TODO:
# normSet = ROOT.RooFit.NormSet(obs)

# TODO:
# wspace.var("x").setRange("x-bin", xmin, xmax)

# TODO:
# integral = model.createIntegral(obs, normSet, ROOT.RooFit.Range("x-bin"))

# 2) Loop over bins and accumulate X, Y
dx = float(xmax - xmin) / M
X, Y = 0.0, 0.0
total = 0.0  # sanity check: Σ n_i should be ~ T
K = 0         # number of bins used (N_i > 5)

print("\n")
print("%5s\t%10s %5s %10s" % ("bin", "binlow", "count", "mean"))

for ii in range(M):
    # Observed count N_i from the RooDataHist
    ibin = hdata.get(ii)   # selects bin ii (needed before weight())
    Ni   = hdata.weight()  # bin content (count)

    # Bin edges [x, x+dx] and integral over that range
    x = xmin + ii * dx
    # TODO:
    # wspace.var("x").setRange("x-bin", x, x + dx)

    # TODO:
    # ni = T * integral.getVal()

    # TODO:
    # print("{:5d}\t{:10.3f} {:5d} {:10.1f}".format(ii + 1, x, int(Ni), ni))

    # TODO:
    # total += ni

    # Use only bins with N_i > 5
    if Ni < 5:
        continue

    # TODO:
    # K += 1
    # X += (Ni - ni)**2 / ni
    # Y += Ni * math.log(ni / Ni)

# Finish Y
# TODO:
# Y *= -2

# 3) Convert to p-values via χ^2 tail probabilities
ndf = K - P - 1

if ndf > 0:
    # TODO:
    # pvalueX = ROOT.TMath.Prob(X, ndf)
    # pvalueY = ROOT.TMath.Prob(Y, ndf)
    pass
else:
    pvalueX, pvalueY = 1.0, 1.0

print("=" * 80)
# TODO:
# print("Int p(x|xi) dx ={:6.1f}\n".format(total))
# print("ChiSq/ndf = {:6.1f}/{:d} (using X)".format(X, ndf))
# print("p-value   = {:9.4f}\n".format(pvalueX))
# print("ChiSq/ndf = {:6.1f}/{:d} (using Y)".format(Y, ndf))
# print("p-value   = %9.4f" % pvalueY)